# Recreate Wiki Collection with nomic-embed-text

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")

print(f"Weaviate Key: {WEAVIATE_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")
print(f"Embedding Model: {EMBEDDING_MODEL}")

Weaviate Key: root-user-key
OpenAI URL: http://host.docker.internal:11434
Embedding Model: nomic-embed-text


In [15]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure

# Connect to Weaviate
client = weaviate.connect_to_local(
    host="localhost",
    port=8080,
    grpc_port=50051,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-OpenAI-BaseURL": OPENAI_URL
    }
)

print(f"Connected: {client.is_ready()}")

Connected: True


/Users/scottaskinosie/Documents/weaviate/workshops/nato/weaviate-workshop/.venv/lib/python3.11/site-packages/weaviate/warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/var/folders/6n/_19_vgbs0yg4thgn5btfcmz80000gn/T/ipykernel_13573/1174873923.py:6: ResourceWarning: unclosed <socket.socket fd=75, family=30, type=1, proto=6, laddr=('::1', 63965, 0, 0), raddr=('::1', 8080, 0, 0)>
  client = weaviate.connect_to_local(


In [ ]:
# Delete existing Wiki collection
try:
    client.collections.delete("Wikinew")
    print("✓ Deleted existing Wiki collection")
except Exception as e:
    print(f"Collection didn't exist or error: {e}")

✓ Deleted existing Wiki collection


In [16]:
client.collections.create(
    name="TestCollection",
    
    # Using local ollama embedding model
    vector_config=Configure.Vectors.text2vec_ollama(
        model="nomic-embed-text",
        api_endpoint=OPENAI_URL
    ),
    
    # Using local ollama generative model
    generative_config=Configure.Generative.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL
    ),
)

UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name TestCollection already exists'}]}.

In [17]:
# Create new collection with nomic-embed-text
collection = client.collections.create(
    name="Wikinew",
    # Using local ollama embedding model
    vector_config=Configure.Vectors.text2vec_ollama(
        model="nomic-embed-text",
        api_endpoint=OPENAI_URL
    ),
    
    # Using local ollama generative model
    generative_config=Configure.Generative.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL
    ),
)

print("✓ Created new Wiki collection with nomic-embed-text")

✓ Created new Wiki collection with nomic-embed-text


In [18]:
# Load data and import
from datasets import load_dataset
from tqdm import tqdm
from weaviate.util import generate_uuid5

print("Loading data...")
dataset = load_dataset('parquet',
                      data_files={'train': ['wiki-data/weaviate/snowflake-arctic-v2/*.parquet']},
                      split="train",
                      streaming=False)

print(f"Loaded {len(dataset)} items")

# Import data (let Weaviate vectorize)
print("Importing and vectorizing...")
with collection.batch.fixed_size(batch_size=200, concurrent_requests=2) as batch:
    for item in tqdm(dataset, desc="Importing"):
        # Only text fields, no vectors
        data_obj = {
            "title": item["title"],
            "text": item["text"],
            "wiki_id": item["wiki_id"],
            "url": item["url"]
        }
        
        id = generate_uuid5(item["wiki_id"])
        batch.add_object(data_obj, uuid=id)
        
        if batch.number_errors > 10:
            print(f"Too many errors: {batch.number_errors}")
            break

print(f"✓ Import completed. Collection count: {len(collection)}")

Loading data...
Loaded 250000 items
Importing and vectorizing...


Importing:   0%|          | 999/250000 [00:07<29:55, 138.68it/s] 


KeyboardInterrupt: 

In [10]:
client.close()
print("✓ Done!")

✓ Done!
